In [4]:
import os
import glob
import cv2
import math
import itertools

import numpy as np
from datetime import datetime
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix

from keras.preprocessing import image
from keras import backend
from keras.models import Sequential, Model, load_model
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers import Conv2D, Input, concatenate
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions, preprocess_input, _obtain_input_shape
from keras.applications import InceptionV3

from keras.utils.data_utils import get_file

import pandas as pd

In [5]:
class DriverDistractionHelper:
    
    def __init__(self, img_rows, img_cols, color_type=1):
        x_train, y_train, driver_id, unique_drivers = self.read_and_normalize_train_data(img_rows, img_cols, color_type)
        x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=4484, random_state=57, stratify=y_train)
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=4484, random_state=57, stratify=y_train)
        
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.x_val = x_val
        self.y_val = y_val
            
    def read_and_normalize_train_data(self, img_rows, img_cols, color_type):
        x_train, y_train, driver_id, unique_drivers = self.load_train(img_rows, img_cols, color_type)
        x_train = np.array(x_train, dtype=np.uint8)
        y_train = np.array(y_train, dtype=np.uint8)
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, color_type)
        y_train = np_utils.to_categorical(y_train, 10)
        
        print('Train shape:', x_train.shape)
        print(x_train.shape[0], 'train samples')
        return x_train, y_train, driver_id, unique_drivers
    
    def read_and_normalize_test_data(self, begin_index, img_rows, img_cols, color_type=1, count=1000):
        test_data, test_ids = self.load_test(begin_index, img_rows, img_cols, color_type, count)

        test_data = np.array(test_data, dtype=np.uint8)
        test_data = test_data.reshape(test_data.shape[0], img_rows, img_cols, color_type)
        
        print('Test shape:', test_data.shape)
        print(test_data.shape[0], 'test samples')
        return test_data, test_ids
    
    def load_train(self, img_rows, img_cols, color_type=1):
        X_train = []
        y_train = []
        driver_id = []

        driver_data = self.get_driver_data()

        print('Read train images')
        for j in range(10):
            print('Load folder c{}'.format(j))
            path = os.path.join('..', 'input', 'train', 'c' + str(j), '*.jpg')
            files = glob.glob(path)
            for fl in files:
                flbase = os.path.basename(fl)
                img = self.get_image(fl, img_rows, img_cols, color_type)
                X_train.append(img)
                y_train.append(j)
                driver_id.append(driver_data[flbase])

        unique_drivers = sorted(list(set(driver_id)))
        print('Unique drivers: {}'.format(len(unique_drivers)))
        print(unique_drivers)
        return X_train, y_train, driver_id, unique_drivers
    
    def load_test(self, begin_index, img_rows, img_cols, color_type=1, count=1000):
        print('Read manual test images')
        path = os.path.join('..', 'input', 'test', '*.jpg')
        files = sorted(glob.glob(path))
        
        if(len(files)-count < begin_index):
            files = files[begin_index : len(files)]
        else:
            files = files[begin_index : begin_index+count]

        X_test = []
        X_test_id = []
        total = 0
        for fl in files:
            flbase = os.path.basename(fl)
            img = self.get_image(fl, img_rows, img_cols, color_type)
            X_test.append(img)
            X_test_id.append(flbase)
            total += 1
        return X_test, X_test_id
    
    def cache_data(self, data, path):
        if os.path.isdir(os.path.dirname(path)):
            file = open(path, 'wb')
            pickle.dump(data, file)
            file.close()
        else:
            print('Directory doesnt exists')
    
    def get_driver_data(self):
        dr = dict()
        path = os.path.join('..', 'input', 'driver_imgs_list.csv')
        print('Read drivers data')
        f = open(path, 'r')
        line = f.readline()
        while (1):
            line = f.readline()
            if line == '':
                break
            arr = line.strip().split(',')
            dr[arr[2]] = arr[0]
        f.close()
        return dr
    
    def get_image(self, path, img_rows, img_cols, color_type=1):        
        # Load as grayscale
        if color_type == 1:
            img = image.load_img(path, grayscale=True, target_size=(img_rows, img_cols))
        elif color_type == 3:
            img = image.load_img(path, target_size=(img_rows, img_cols))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = self.preprocess_input(img)
        
        return img

    def preprocess_input(self, img):
        img /= 255.
        img -= 0.5
        img *= 2.
        return img

In [9]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues, filename=None):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    if not filename == None:
        plt.savefig(filename)
        plt.close()

In [10]:
img_rows = 299
img_cols = 299
color_type = 3

batch_size = 32
nb_epoch = 4
random_state = 51

In [11]:
ddh = DriverDistractionHelper(img_rows, img_cols, color_type)

Read drivers data
Read train images
Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9
Unique drivers: 26
['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
('Train shape:', (22424, 299, 299, 3))
(22424, 'train samples')


In [12]:
from collections import Counter
print Counter(np.argmax(ddh.y_test, axis=1))
print Counter(np.argmax(ddh.y_val, axis=1))
print Counter(np.argmax(ddh.y_train, axis=1))

Counter({0: 498, 3: 469, 4: 465, 6: 465, 2: 463, 5: 462, 1: 453, 9: 426, 7: 401, 8: 382})
Counter({0: 498, 3: 469, 4: 465, 6: 465, 2: 463, 5: 462, 1: 454, 9: 426, 7: 400, 8: 382})
Counter({0: 1493, 3: 1408, 4: 1396, 6: 1395, 2: 1391, 5: 1388, 1: 1360, 9: 1277, 7: 1201, 8: 1147})


In [6]:
def create_model(img_rows, img_cols, color_type=1, num_classes=10):
    model = load_model('models/inceptionV3_model.h5')
    return model

In [7]:
classes = ['safe driving', 'texting - right', 'talking on the phone - right', 'exting - left', 'talking on the phone - left',
          'operating the radio', 'drinking', 'reaching behind', 'hair and makeup', 'talking to passenger']
keys = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [13]:
true_y_test = np.argmax(ddh.y_test, axis=1)

In [15]:
model = create_model(img_rows=img_rows, img_cols=img_cols, color_type=color_type, num_classes=10)

In [16]:
model.fit(ddh.x_train, ddh.y_train, batch_size=batch_size, epochs=1, verbose=1, validation_data=(ddh.x_val, ddh.y_val))
scores = model.evaluate(x=ddh.x_test, y=ddh.y_test, batch_size=128)
print 'Metrics: ', scores
y_pred_test = model.predict(ddh.x_test, batch_size=128, verbose=1)
y_pred_test = np.argmax(y_pred_test, axis=1)

cm = confusion_matrix(true_y_test, y_pred_test)
plot_confusion_matrix(cm, keys, normalize=False, filename='cm_epoch_'+str(9))
print str(9), ': <======Finished======>'

Train on 13456 samples, validate on 4484 samples
Epoch 1/1
4484/4484 [==============================] - 1135s 253ms/step
Metrics:  [0.0606774229044967, 0.9830508474576272]
4484/4484 [==============================] - 1137s 254ms/step
9 : <======Finished======>


In [17]:
model.save('models/inceptionV3_model.h5')
model.save_weights('models/inceptionV3_model_weights.h5')